<a href="https://colab.research.google.com/github/RAYYAN-HUB/rayyan/blob/main/Copy_of_breast_cancer_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Breast CANCER PREDICTION USING MACHINE LEARNING


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install pydicom

In [ ]:
#importing libraries
from tensorflow import keras
import numpy as np
from tensorflow.keras.preprocessing.image import load_img
import os
import pandas as pd
import pydicom as dicom
from skimage.transform import resize
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
dataset_path="/content/drive/MyDrive/INbreast"
train_df_path=dataset_path+"/Train_INbreast.csv"
test_df_path=dataset_path+"/Test_INbreast.csv"

train_df = pd.read_csv(train_df_path)
test_df = pd.read_csv(test_df_path)

In [ ]:
print(f"Total train dataset: {len(train_df)}")
train_df.head()

In [ ]:
print(f"Total test dataset: {len(test_df)}")
test_df.head()

In [ ]:
train_df.tail()

In [ ]:
test_df.tail()

In [ ]:
#return the size of data
traindata = np.zeros((224, 224, 3))
print(traindata.shape)

In [ ]:
print(train_df.columns)

In [ ]:
df = pd.DataFrame(train_df)

sns.pairplot(df.iloc[:, 1:7])

In [ ]:
plt.figure(figsize=(10, 10))
sns.heatmap(df.iloc[:,1:32].corr(),annot=True,fmt=".0%")

In [ ]:
class ImageDataset(keras.utils.Sequence):
    """Helper to iterate over the data (as Numpy arrays)."""

    def __init__(self, root_dir, transform = None, train = True):
        self.root_dir = root_dir
        self.transform = transform
        #self.paths = os.listdir(self.image_dir)
        self.train = train
        if self.train:
            self.labels = pd.read_csv(self.root_dir + 'Train_INbreast.csv', delimiter = ',')
            self.labels = self.labels['Bi-Rads']
            self.image_dir = self.root_dir + 'Train_AllDICOMs'
            self.paths = os.listdir(self.image_dir)
        else:
            self.labels = pd.read_csv(self.root_dir + 'Val_INbreast.csv', delimiter = ',')
            self.labels = self.labels['Bi-Rads']
            self.image_dir = self.root_dir + 'Val_AllDICOMs'
            self.paths = os.listdir(self.image_dir)

    

    def __len__(self):
        return len(self.paths)
    
    def flip(self, image, vflip=False, hflip=False):
        '''
        Flip the image
        :param image: image to be processed
        :param vflip: whether to flip the image vertically
        :param hflip: whether to flip the image horizontally
        '''
        if hflip or vflip:
            if hflip and vflip:
                c = -1
            else:
                c = 0 if vflip else 1
            image = cv2.flip(image, flipCode=c)
        return image 
    
    def rotate(self, image, angle=90, scale=1.0):
        '''
        Rotate the image
        :param image: image to be processed
        :param angle: Rotation angle in degrees. Positive values mean counter-clockwise rotation (the coordinate origin is assumed to be the top-left corner).
        :param scale: Isotropic scale factor.
        '''
        w = image.shape[1]
        h = image.shape[0]
        #rotate matrix
        M = cv2.getRotationMatrix2D((w/2,h/2), angle, scale)
        #rotate
        image = cv2.warpAffine(image,M,(w,h))
        return image
    
    def invert_image(self, image,channel=1):
        image=(channel-image)
        return image
        
    def gausian_blur(self, image,blur = 0.3):
        image = cv2.GaussianBlur(image,(5,5),blur)
        return image
    

    def augment(self, image):
       frame = self.rotate(image, angle=90, scale=1.0)
       frame = self.flip(image, vflip=True, hflip=False)
       frame = self.flip(image, vflip=False, hflip=True)
       frame = self.invert_image(frame)
       frame = self.gausian_blur(frame)
       return frame

    def __getitem__(self, index):
        array = dicom.dcmread(self.image_dir+'/'+self.paths[index], force=True)
        array.file_meta.TransferSyntaxUID = dicom.uid.ImplicitVRLittleEndian
        array = array.pixel_array.astype('float')
        #array = cv2.resize(frame, (224, 224), interpolation = cv2.INTER_AREA)
        array = resize(array, (224, 224), anti_aliasing=True)
        label = list(self.labels[index])[0]
        label = np.asarray([int(label)])
        label = tf.keras.utils.to_categorical(label -1, num_classes=6)
        
        
        if self.transform:
            array = self.augment(array)
            array = np.stack((array,)*3, axis=2)
            array = np.expand_dims(array, axis=0)
            array = np.asarray(array)
            
        else:
            array = np.stack((array,)*3, axis=2)
            array = np.expand_dims(array, axis=0)
            array = np.asarray(array)
        return array, label

In [ ]:
train_gen = ImageDataset('/content/drive/MyDrive/INbreast/', transform = True, train = True)

In [ ]:
val_gen = ImageDataset('/content/drive/MyDrive/INbreast/', transform = None, train = False)

In [ ]:
for idx, (x, y) in enumerate(val_gen):
  print(idx, x.shape, y.shape)
  break

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16

base_model = VGG16(input_shape = (224, 224, 3), # Shape of our images
include_top = False, # Leave out the last fully connected layer
weights = 'imagenet')

In [ ]:
# Flatten the output layer to 1 dimension
x = tf.keras.layers.Flatten()(base_model.output)

# Add a fully connected layer with 512 hidden units and ReLU activation
x = tf.keras.layers.Dense(512, activation='relu')(x)

# Add a dropout rate of 0.5
x = tf.keras.layers.Dropout(0.5)(x)

# Add a final sigmoid layer with 1 node for classification output
x = tf.keras.layers.Dense(6, activation='softmax')(x)

model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.00001), loss = 'CategoricalCrossentropy',metrics = ['AUC'])

In [ ]:
checkpoint_filepath = "/content/drive/MyDrive/INBreastNet/INBreast/Model.h5"
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_auc',
    mode='max',
    save_best_only=True,
    verbose=1)

vgghist = model.fit((train_gen), shuffle=True, batch_size = 8, epochs = 10, callbacks=[model_checkpoint_callback], validation_data=(val_gen))

In [ ]:
class TestDataset(keras.utils.Sequence):
    """Helper to iterate over the data (as Numpy arrays)."""

    def __init__(self, root_dir):
        self.root_dir = root_dir
        self.image_dir = self.root_dir + 'Val_AllDICOMs'
        self.paths = os.listdir(self.image_dir)

    

    def __len__(self):
        return len(self.paths)
    
    

    def __getitem__(self, index):
        array = dicom.dcmread(self.image_dir+'/'+self.paths[index], force=True)
        array.file_meta.TransferSyntaxUID = dicom.uid.ImplicitVRLittleEndian
        array = array.pixel_array.astype('float')
        #array = cv2.resize(frame, (224, 224), interpolation = cv2.INTER_AREA)
        array = resize(array, (224, 224), anti_aliasing=True)

        array = np.stack((array,)*3, axis=2)
        array = np.expand_dims(array, axis=0)
        array = np.asarray(array)
        return array

In [ ]:
test_gen = TestDataset('/content/drive/MyDrive/INbreast/')

In [ ]:
model = keras.models.load_model('/content/drive/MyDrive/INBreastNet/INBreast/Model.h5')

In [ ]:
y_prob = model.predict((test_gen))
y_classes = y_prob.argmax(axis=1)
y_classes